Dataset:
https://www.kaggle.com/quora/question-pairs-dataset


Quora gets lot of duplicate questions which is added by it’s user from different locations and the main intent of Quora is to have a unique questions which can be answered by other users who are an expert or provide their opinion about the Question being asked. The primary goal of this competition is to go through the pair of questions and identify whether they are identical or not. For example, the queries “What is the most populous state in the USA?” and “Which state in the United States has the most people?” should not exist separately on Quora because the intent behind both is identical

In [ ]:
import tensorflow as tf
import pandas as pd
import numpy as np
import nltk
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer
import re
from gensim import utils
from gensim.models.doc2vec import LabeledSentence
from gensim.models.doc2vec import TaggedDocument
from gensim.models import Doc2Vec
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics import accuracy_score

# Import Data

df = pd.read_csv('questions.csv')


In [ ]:
# dff=df.dropna()
#nullvaule = df[df.isnull().any(axis=1)] # check for null value
# ab=df.drop(nullvaule.index, inplace=True)
# dff=df.update(ab)



In [ ]:
dir(pd)

In [ ]:
first=df.loc[:,('question1')]
print(type(first))
first.dropna(axis=0)
print(type(first))
print(first[363417])
len(first)
first
#first2=df.loc[:,('question2')]

<class 'pandas.core.series.Series'>
<class 'pandas.core.series.Series'>
Who invested in mycraftycloset.com?


0         What is the step by step guide to invest in sh...
1         What is the story of Kohinoor (Koh-i-Noor) Dia...
2         How can I increase the speed of my internet co...
3         Why am I mentally very lonely? How can I solve...
4         Which one dissolve in water quikly sugar, salt...
                                ...                        
404346    How many keywords are there in the Racket prog...
404347            Do you believe there is life after death?
404348                                    What is one coin?
404349    What is the approx annual cost of living while...
404350                What is like to have sex with cousin?
Name: question1, Length: 404350, dtype: object

In [ ]:

question1token=[]
for index, value in first.items():
#for txt in first1.iteritems():
  print(f"Index : {index}, Value : {value}")
  ob= nltk.word_tokenize(first[index])
  question1token.append(ob)
set(question1token)


In [ ]:
question1token

[['What',
  'is',
  'the',
  'step',
  'by',
  'step',
  'guide',
  'to',
  'invest',
  'in',
  'share',
  'market',
  'in',
  'india',
  '?'],
 ['What',
  'is',
  'the',
  'story',
  'of',
  'Kohinoor',
  '(',
  'Koh-i-Noor',
  ')',
  'Diamond',
  '?'],
 ['How',
  'can',
  'I',
  'increase',
  'the',
  'speed',
  'of',
  'my',
  'internet',
  'connection',
  'while',
  'using',
  'a',
  'VPN',
  '?'],
 ['Why',
  'am',
  'I',
  'mentally',
  'very',
  'lonely',
  '?',
  'How',
  'can',
  'I',
  'solve',
  'it',
  '?'],
 ['Which',
  'one',
  'dissolve',
  'in',
  'water',
  'quikly',
  'sugar',
  ',',
  'salt',
  ',',
  'methane',
  'and',
  'carbon',
  'di',
  'oxide',
  '?'],
 ['Astrology',
  ':',
  'I',
  'am',
  'a',
  'Capricorn',
  'Sun',
  'Cap',
  'moon',
  'and',
  'cap',
  'rising',
  '...',
  'what',
  'does',
  'that',
  'say',
  'about',
  'me',
  '?'],
 ['Should', 'I', 'buy', 'tiago', '?'],
 ['How', 'can', 'I', 'be', 'a', 'good', 'geologist', '?'],
 ['When', 'do', 'you', '

In [ ]:

from __future__ import print_function
import os
import re
import tqdm
import string
import pandas as pd
import numpy as np
import keras
sentences=[]
for index, row in df['question1'].iteritems():
  sentences.append(LabeledSentence(utils.to_unicode(row).split(), ['question1' + '_%s' % str(index)]))



In [ ]:
print(sentences)

In [ ]:
df.tail()
d=df.loc[:,('question1','question2')]
question1token=[]
ob=nltk.word_tokenize(df['question1'][0])
print(ob)
abb = df['question1']
print(abb)
for i in abb[i] : 
  ob=nltk.word_tokenize(df['question1'][i])
  question1token.extend(ob)

In [ ]:
df.tail()
d=df.loc[:,('question1','question2')]
question1token=[]
ob=nltk.word_tokenize(df['question1'][0])
print(ob)
abb = df['question1']
print(abb)
for i in abb[i] : 
  ob=nltk.word_tokenize(df['question1'][i])
  question1token.extend(ob)

In [ ]:
df

(404351, 6)

In [ ]:
df.describe()

,id,qid1,qid2,is_duplicate
count,404351.000000,404351.000000,404351.000000,404351.000000
mean,202175.000000,391840.987691,390195.973765,0.369248
std,116726.223686,228430.857607,228803.645742,0.482602
min,0.000000,1.000000,2.000000,0.000000
25%,101087.500000,193381.000000,191012.000000,0.000000
50%,202175.000000,390630.000000,388364.000000,0.000000
75%,303262.500000,589514.000000,588071.000000,1.000000
max,404350.000000,789800.000000,789801.000000,1.000000


In [ ]:
df.head(10)

,id,qid1,qid2,question1,question2,is_duplicate
0,0,1,2,What is the step by step guide to invest in sh...,What is the step by step guide to invest in sh...,0
1,1,3,4,What is the story of Kohinoor (Koh-i-Noor) Dia...,What would happen if the Indian government sto...,0
2,2,5,6,How can I increase the speed of my internet co...,How can Internet speed be increased by hacking...,0
3,3,7,8,Why am I mentally very lonely? How can I solve...,Find the remainder when [math]23^{24}[/math] i...,0
4,4,9,10,"Which one dissolve in water quikly sugar, salt...",Which fish would survive in salt water?,0
5,5,11,12,Astrology: I am a Capricorn Sun Cap moon and c...,"I'm a triple Capricorn (Sun, Moon and ascendan...",1
6,6,13,14,Should I buy tiago?,What keeps childern active and far from phone ...,0
7,7,15,16,How can I be a good geologist?,What should I do to be a great geologist?,1
8,8,17,18,When do you use シ instead of し?,"When do you use ""&"" instead of ""and""?",0
9,9,19,20,Motorola (company): Can I hack my Charter Moto...,How do I hack Motorola DCX3400 for free internet?,0


In [ ]:
# check  for null values
nv=df[df.isnull().any(axis=1)]
# drop row with null values
df.drop(nv.index,inplace=True)

labeled_questions=[]
labeled_questions.append(TaggedDocument(d['questions1'][i].split(), df[df.index == i].qid1))
labeled_questions.append(TaggedDocument(df[questions2[i].split()], df[df.index == i].qid2))

In [ ]:
#Remove Stop words
if remove_stopwords:
  stops = set(stopwords.words("english"))
  words = [w for w in text.lower().split() if not w in stops]

final_text = " ".join(words)